In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import platform

# Paths to the dataset directories
train_dir = "/Users/rukmini/Documents/Project/newdata/reduced_train"
test_dir = "/Users/rukmini/Documents/Project/newdata/reduced_test"


# Parameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = len([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
epochs = 3

# Enable eager execution (if necessary)
tf.config.run_functions_eagerly(True)

# Data augmentation and preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generators for training and testing
train_generator = data_gen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Ensure categorical matches with loss
)

test_generator = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Ensure categorical matches with loss
    shuffle=False
)

# Validate generator output shapes
assert train_generator.image_shape == (img_height, img_width, 3), "Train images must be RGB"
assert test_generator.image_shape == (img_height, img_width, 3), "Test images must be RGB"

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model

# Build the model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Ensure num_classes matches the dataset
])

# Compile the model
if platform.system() == "Darwin" and platform.processor() == "arm":
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0005)
else:
    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)



model.compile(optimizer=opt, loss='categorical_crossentropy',  # Matches categorical labels
metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

# Evaluate the model
eval_results = model.evaluate(test_generator)
print(f"Test Loss: {eval_results[0]}, Test Accuracy: {eval_results[1]}")

# Generate classification report and confusion matrix
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

print("Confusion Matrix:\n")
print(confusion_matrix(y_true, y_pred))

# Save the model
model.save('vgg_ucf_model.h5')




Found 6309 images belonging to 14 classes.
Found 557 images belonging to 14 classes.
Epoch 1/3
198/198 [==============================] - 7901s 40s/step - loss: 1.2859 - accuracy: 0.7393 - val_loss: 1.8341 - val_accuracy: 0.5817
Epoch 2/3
198/198 [==============================] - 7983s 40s/step - loss: 1.0932 - accuracy: 0.7515 - val_loss: 1.8321 - val_accuracy: 0.5817
Epoch 3/3
18/18 [==============================] - 206s 11s/step - loss: 1.7576 - accuracy: 0.5817
Test Loss: 1.7575714588165283, Test Accuracy: 0.5816876292228699
18/18 [==============================] - 199s 11s/step
Classification Report:

               precision    recall  f1-score   support

        Abuse       0.00      0.00      0.00         1
       Arrest       0.00      0.00      0.00        17
        Arson       0.00      0.00      0.00        15
      Assault       0.00      0.00      0.00        14
     Burglary       0.00      0.00      0.00        42
    Explosion       0.00      0.00      0.00        3

/Users/rukmini/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rukmini/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rukmini/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rukmini/anacond

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import platform

# Paths to the dataset directories
train_dir = "/Users/rukmini/Documents/Project/newdata/reduced_train"
test_dir = "/Users/rukmini/Documents/Project/newdata/reduced_test"


# Parameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = len([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
epochs = 10

# Enable eager execution (if necessary)
tf.config.run_functions_eagerly(True)

# Data augmentation and preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generators for training and testing
train_generator = data_gen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Ensure categorical matches with loss
)

test_generator = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Ensure categorical matches with loss
    shuffle=False
)

# Validate generator output shapes
assert train_generator.image_shape == (img_height, img_width, 3), "Train images must be RGB"
assert test_generator.image_shape == (img_height, img_width, 3), "Test images must be RGB"

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model

# Build the model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Ensure num_classes matches the dataset
])

# Compile the model
if platform.system() == "Darwin" and platform.processor() == "arm":
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0005)
else:
    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)



model.compile(optimizer=opt, loss='categorical_crossentropy',  # Matches categorical labels
metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

# Evaluate the model
eval_results = model.evaluate(test_generator)
print(f"Test Loss: {eval_results[0]}, Test Accuracy: {eval_results[1]}")

# Generate classification report and confusion matrix
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

print("Confusion Matrix:\n")
print(confusion_matrix(y_true, y_pred))

# Save the model
model.save('vgg_ucf_model.h5')




Found 126445 images belonging to 14 classes.
Found 11046 images belonging to 14 classes.
Epoch 1/10
  25/3952 [..............................] - ETA: 43:29:54 - loss: 2.0592 - accuracy: 0.6700

KeyboardInterrupt: 